In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))

1. 评估函数
2. keras 实现TextCNN 多分类
3. keras 实现TextCNN 多标签分类

In [174]:
import time
import numpy as np
import tensorflow as tf
from utils.preprocess import load_testcnn_data, load_tokenizer_binarizer
tf.__version__

'2.0.0'

**加载数据**

In [5]:
train_x, dev_x, test_x, train_y, dev_y, test_y = load_testcnn_data()

## TextCNN详细过程：
![](../images/TextCNN.png)

+ Embedding：第一层是图中最左边的7乘5的句子矩阵，每行是词向量，维度=5，这个可以类比为图像中的原始像素点。
+ Convolution：然后经过 kernel_sizes=(2,3,4) 的一维卷积层，每个kernel_size 有两个输出 channel。
+ MaxPolling：第三层是一个1-max pooling层，这样不同长度句子经过pooling层之后都能变成定长的表示。
+ FullConnection and Softmax：最后接一层全连接的 softmax 层，输出每个类别的概率。


### 通道（Channels）：

+ 图像中可以利用 (R, G, B) 作为不同channel；
+ 文本的输入的channel通常是不同方式的embedding方式（比如 word2vec或Glove），实践中也有利用静态词向量和fine-tunning词向量作为不同channel的做法。
 

### 一维卷积（conv-1d）：
![](../images/conv1D.png)
+ 图像是二维数据；
+ 文本是一维数据，因此在TextCNN卷积用的是一维卷积（在word-level上是一维卷积；虽然文本经过词向量表达后是二维数据，但是在embedding-level上的二维卷积没有意义）。一维卷积带来的问题是需要通过设计不同 kernel_size 的 filter 获取不同宽度的视野。

### 特征：词向量表示方式

+ 数据量较大：可以直接随机初始化embeddings，然后基于语料通过训练模型网络来对embeddings进行更新和学习。
+ 数据量较小：可以利用外部语料来预训练(pre-train)词向量，然后输入到Embedding层，用预训练的词向量矩阵初始化embeddings。（通过设置weights=[embedding_matrix]）。
+ 静态(static)方式：训练过程中不再更新embeddings。实质上属于迁移学习，特别是在目标领域数据量比较小的情况下，采用静态的词向量效果也不错。（通过设置trainable=False）
+ 非静态(non-static)方式：在训练过程中对embeddings进行更新和微调(fine tune)，能加速收敛。（通过设置trainable=True）

## 使用类似tensorflow官方教程的方式建模

In [227]:
import tensorflow as tf

In [207]:
class TextCNN(tf.keras.Model):
    def __init__(self, max_len, vocab_size, embedding_dim, output_dim, 
                 kernel_sizes, filters=2, embedding_matrix=None):
        super(TextCNN, self).__init__()
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        
        if embedding_matrix is None:
            self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        else:
            self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                        weights=[embedding_matrix], trainable=True)
            
        self.kernel_sizes = kernel_sizes  # exm: [2, 3, 4]
        self.filters = filters
        self.conv1d = [tf.keras.layers.Conv1D(filters=self.filters, kernel_size=k, strides=1) 
                       for k in self.kernel_sizes]
        
        self.maxpool1d = [tf.keras.layers.MaxPool1D(max_len-k+1) for k in self.kernel_sizes]
        
        # 输出层
        self.dense = tf.keras.layers.Dense(output_dim)
        
    def call(self, x):
        x = self.embedding(x)
        pool_output = []
        for conv, pool in zip(self.conv1d, self.maxpool1d):
            c = conv(x)  # (batch_size, max_len-kernel_size+1, filters)
            p = pool(c)  # (batch_size, 1, filters)
            pool_output.append(p)

        pool_output = tf.concat(pool_output, axis=2)  # (batch_size, 1, n*filters)
        pool_output = tf.squeeze(pool_output)  # (batch_size, n*filters)
        y = self.dense(pool_output)
        return y

### 测试模型

In [208]:
model = TextCNN(max_len=128, 
                vocab_size=50000, 
                embedding_dim=256, 
                output_dim=97, 
                kernel_sizes=[2, 3, 4])

In [209]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(100)
train_dataset = train_dataset.batch(32, drop_remainder=True)
x, y = next(iter(train_dataset))

In [210]:
model(x).shape

TensorShape([32, 97])

### 参数设置和训练

In [236]:
max_len = 128
vocab_size = 50000
embedding_dim=256
output_dim = len(train_y[0])  # 97
kernel_sizes = [2, 3, 4]
learning_rate = 0.001
batch_size = 32
epochs = 5
steps_per_epoch = len(train_x) // batch_size

In [237]:
model = TextCNN(max_len=max_len, 
                vocab_size=vocab_size, 
                embedding_dim=embedding_dim, 
                output_dim=output_dim, 
                kernel_sizes=kernel_sizes)

In [238]:
# loss_object = tf.keras.losses.CategoricalCrossentropy()  # 多分类
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)  # 多标签分类
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [239]:
loss_record = []
for epoch in range(epochs):
    start = time.time()
    total_loss = 0
    
    for batch, (x, y) in enumerate(train_dataset.take(steps_per_epoch)):
        with tf.GradientTape() as tape:
            y_pred = model(x)
            loss = loss_object(y, y_pred)
        total_loss += loss
        
        variables = model.trainable_variables
        grads = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(grads, variables))
        if batch % 100 == 0:
            print('epoch {:2d} batch {:4d} loss {:.4f}'.format(epoch+1, batch+1, loss.numpy()))
    
    loss_record.append(total_loss)  # 记录下每轮loss的变化
    print('Epoch {:2d} Loss {:.4f}'.format(epoch + 1,
                                       total_loss / steps_per_epoch))
    
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))


epoch  1 batch    1 loss 0.6929
epoch  1 batch  101 loss 0.1906
epoch  1 batch  201 loss 0.1719
epoch  1 batch  301 loss 0.1419
epoch  1 batch  401 loss 0.1455
epoch  1 batch  501 loss 0.1591
Epoch  1 Loss 0.2070
Time taken for 1 epoch 190.80426454544067 sec

epoch  2 batch    1 loss 0.1326
epoch  2 batch  101 loss 0.1161
epoch  2 batch  201 loss 0.1166
epoch  2 batch  301 loss 0.1084
epoch  2 batch  401 loss 0.0871
epoch  2 batch  501 loss 0.0891
Epoch  2 Loss 0.1071
Time taken for 1 epoch 179.6209602355957 sec

epoch  3 batch    1 loss 0.0825
epoch  3 batch  101 loss 0.0833
epoch  3 batch  201 loss 0.0752
epoch  3 batch  301 loss 0.0790
epoch  3 batch  401 loss 0.0597
epoch  3 batch  501 loss 0.0861
Epoch  3 Loss 0.0749
Time taken for 1 epoch 190.95905780792236 sec

epoch  4 batch    1 loss 0.0718
epoch  4 batch  101 loss 0.0629
epoch  4 batch  201 loss 0.0614
epoch  4 batch  301 loss 0.0621
epoch  4 batch  401 loss 0.0713
epoch  4 batch  501 loss 0.0731
Epoch  4 Loss 0.0630
Time tak

### 预测

In [230]:
from sklearn.metrics import f1_score, accuracy_score, classification_report

In [231]:
y_pred = model(test_x)
y_pred = np.where(y_pred>0.5,1,0)
tokenizer, mlb = load_tokenizer_binarizer()

In [232]:
mlb.inverse_transform(y_pred)

[('政治',),
 ('地理', '宇宙中的地球'),
 ('人口与城市', '人口增长与人口问题', '人口迁移与人口流动', '地理'),
 ('人工授精、试管婴儿等生殖技术', '减数分裂与有丝分裂的比较', '生物', '生物性污染', '生物科学与社会', '避孕的原理和方法'),
 ('历史', '古代史'),
 ('生物', '稳态与环境'),
 ('地理', '宇宙中的地球'),
 ('分子与细胞', '生物'),
 ('人体免疫系统在维持稳态中的作用',
  '体液免疫的概念和过程',
  '免疫系统的功能',
  '免疫系统的组成',
  '内环境的稳态',
  '分子与细胞',
  '生物',
  '稳态与环境'),
 ('地球运动的地理意义', '地理', '宇宙中的地球'),
 (),
 ('历史',),
 ('人工授精、试管婴儿等生殖技术',
  '基因的分离规律的实质及应用',
  '生物',
  '生物性污染',
  '生物科学与社会',
  '遗传的分子基础',
  '遗传的细胞基础',
  '避孕的原理和方法'),
 ('生物',),
 ('生物',),
 ('人工授精、试管婴儿等生殖技术',
  '生物',
  '生物性污染',
  '生物科学与社会',
  '遗传的分子基础',
  '遗传的细胞基础',
  '避孕的原理和方法'),
 ('生物', '生物技术在其他方面的应用'),
 ('生物',),
 ('地理',),
 ('人工授精、试管婴儿等生殖技术',
  '生物',
  '生物性污染',
  '生物科学与社会',
  '遗传的分子基础',
  '遗传的细胞基础',
  '避孕的原理和方法'),
 ('地理', '宇宙中的地球'),
 (),
 ('历史',),
 ('地理',),
 ('人口与城市', '地理'),
 ('地理', '宇宙中的地球'),
 ('基因工程的原理及技术', '基因工程的概念', '复等位基因', '生物', '生物技术在其他方面的应用', '生物技术实践'),
 ('历史',),
 ('人工授精、试管婴儿等生殖技术', '生物', '生物性污染', '避孕的原理和方法'),
 ('地理', '宇宙中的地球'),
 ('生物',),
 ('人工授精、试管婴儿等生殖技术',
  '减数分裂与

In [233]:
mlb.inverse_transform(test_y)

[('公民道德与伦理常识', '政治', '社会主义市场经济的伦理要求'),
 ('地理', '太阳对地球的影响', '宇宙中的地球'),
 ('人口与城市', '人口增长与人口问题', '地理'),
 ('人工授精、试管婴儿等生殖技术', '减数分裂与有丝分裂的比较', '生物', '生物性污染', '生物科学与社会', '避孕的原理和方法'),
 ('历史', '清末民主革命风潮', '近代史'),
 ('人体免疫系统在维持稳态中的作用',
  '体液免疫的概念和过程',
  '免疫系统的功能',
  '免疫系统的组成',
  '内环境的稳态',
  '生物',
  '稳态与环境'),
 ('地球运动的地理意义', '地理', '宇宙中的地球'),
 ('生物', '生物工程技术', '生物技术实践'),
 ('人体免疫系统在维持稳态中的作用',
  '体液免疫的概念和过程',
  '免疫系统的功能',
  '免疫系统的组成',
  '内环境的稳态',
  '生物',
  '稳态与环境'),
 ('地球运动的基本形式', '地理', '宇宙中的地球'),
 ('历史', '古代史', '文艺的春天'),
 ('历史', '近代史'),
 ('人工授精、试管婴儿等生殖技术',
  '伴性遗传',
  '生物',
  '生物性污染',
  '生物科学与社会',
  '遗传的分子基础',
  '遗传的细胞基础',
  '避孕的原理和方法'),
 ('生物', '稳态与环境'),
 ('生物', '生物工程技术', '生物技术实践'),
 ('人工授精、试管婴儿等生殖技术',
  '基因的自由组合规律的实质及应用',
  '生物',
  '生物性污染',
  '生物科学与社会',
  '遗传的分子基础',
  '遗传的细胞基础',
  '避孕的原理和方法'),
 ('培养基与无菌技术', '生物', '生物技术在其他方面的应用', '生物技术实践'),
 ('人体免疫系统在维持稳态中的作用',
  '体液免疫的概念和过程',
  '免疫系统的功能',
  '免疫系统的组成',
  '内环境的稳态',
  '生物',
  '稳态与环境'),
 ('地球与地图', '地理'),
 ('人工授精、试管婴儿等生殖技术',
  '基因的自由组合规律的实质及应用',
  '生物',

In [234]:
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.49      0.64       172
           1       1.00      0.03      0.06       105
           2       0.62      0.07      0.12       148
           3       0.93      0.49      0.64       172
           4       0.75      0.26      0.39       215
           5       0.65      0.12      0.21       139
           6       0.00      0.00      0.00        64
           7       0.00      0.00      0.00        64
           8       0.82      0.64      0.72       319
           9       0.86      0.23      0.37        77
          10       0.12      0.05      0.07        56
          11       0.94      0.93      0.94       912
          12       0.00      0.00      0.00       118
          13       0.65      0.14      0.24       139
          14       0.63      0.14      0.22       139
          15       0.67      0.13      0.22       139
          16       0.97      0.25      0.39       359
          17       1.00    

E:\software\Anaconda3\envs\tf2.0\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
E:\software\Anaconda3\envs\tf2.0\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
